In [1]:
!pip install joblib

import os
import time
import glob
import yaml
import argparse
import logging
import pathlib
import struct
import warnings
from random import randint

import numpy as np
import torch
from tqdm import tqdm
from scipy.spatial import ConvexHull, Delaunay, cKDTree, KDTree
from sklearn.neighbors import NearestNeighbors

import open3d as o3d
import trimesh
import k3d
import wandb

warnings.simplefilter(action='ignore', category=FutureWarning)

DEPRECATION: Loading egg at /sfs/gpfs/tardis/home/rhm4nj/.local/lib/python3.11/site-packages/chamfer-2.0.0-py3.11-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /sfs/gpfs/tardis/home/rhm4nj/.local/lib/python3.11/site-packages/emd_ext-0.0.0-py3.11-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# Load the PLY mesh
room_name = "office_4"
mesh = trimesh.load_mesh(f"/scratch/rhm4nj/cral/datasets/Replica-Dataset/ReplicaSDK/{room_name}/habitat/mesh_preseg_semantic.ply")
labels = np.fromfile(f"/scratch/rhm4nj/cral/datasets/Replica-Dataset/ReplicaSDK/{room_name}/semantic.bin", dtype=np.uint32)

In [ ]:
def downsample_random_indices(points, K):
    indices = np.random.choice(points.shape[0], K, replace=False)
    return indices

def convert_rgb_to_k3d_colors(rgb: np.ndarray, alpha: int = 255) -> np.ndarray:
    rgb = np.asarray(rgb)
    if rgb.max() <= 1.0:
        rgb = (rgb * 255).astype(np.uint8)

    rgba = np.zeros((rgb.shape[0], 4), dtype=np.uint8)
    rgba[:, :3] = rgb
    rgba[:, 3] = alpha

    return rgba[:, 0].astype(np.uint32) << 24 | \
           rgba[:, 1].astype(np.uint32) << 16 | \
           rgba[:, 2].astype(np.uint32) << 8  | \
           rgba[:, 3].astype(np.uint32)

print(mesh.visual.vertex_colors.shape)

feat = np.array(mesh.visual.vertex_colors)[:, :3]  # (N, 3) colors
vertices = np.array(mesh.vertices)  # (N, 3) array of vertex positions
faces = np.array(mesh.faces)  # (M, 3) array of triangle indices
print(feat.shape, vertices.shape, faces.shape, labels.shape)
# print(np.unique(labels))

indices = downsample_random_indices(vertices, 400_000)
point_cloud = vertices[indices]
point_cloud_feats = feat[indices]
point_cloud_colors = convert_rgb_to_k3d_colors(point_cloud_feats)

print(point_cloud_colors.shape)
print(point_cloud.shape, np.unique(point_cloud_colors).shape)


(954491, 4)
(954491, 3) (954491, 3) (1907294, 3) (1906932,)
(400000,)
(400000, 3) (54149,)


In [4]:
import numpy as np
from sklearn.decomposition import PCA

def calculate_bounds(points):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    
    return np.array([[x_min, x_max], [y_min, y_max], [z_min, z_max]]), (np.array([x_min, y_min, z_min]), np.array([x_max, y_max, z_max]))

def partition_point_cloud_grid(points, nx=1, ny=1, nz=1):
    # Get min and max bounds
    min_bounds = np.min(points, axis=0)
    max_bounds = np.max(points, axis=0)

    # Compute edges for each axis
    x_edges = np.linspace(min_bounds[0], max_bounds[0], nx + 1)
    y_edges = np.linspace(min_bounds[1], max_bounds[1], ny + 1)
    z_edges = np.linspace(min_bounds[2], max_bounds[2], nz + 1)

    # Store partitions in a dictionary keyed by (ix, iy, iz)
    partitions = {}
    for ix in range(nx):
        for iy in range(ny):
            for iz in range(nz):
                partitions[(ix, iy, iz)] = []

    # Assign points to their bins
    for point in points:
        ix = min(np.searchsorted(x_edges, point[0], side='right') - 1, nx - 1)
        iy = min(np.searchsorted(y_edges, point[1], side='right') - 1, ny - 1)
        iz = min(np.searchsorted(z_edges, point[2], side='right') - 1, nz - 1)
        partitions[(ix, iy, iz)].append(point)

    # Convert to numpy arrays
    partitions = [np.array(v) for _, v in partitions.items()]
    return partitions

partitions = partition_point_cloud_grid(point_cloud, nx=2, ny=2)

In [6]:
import random

plot = k3d.plot()
# plot += k3d.points(point_cloud, point_size=0.01)
for ps in partitions:
    print(ps.shape)
    color = (random.randint(0, 255) << 16) + (random.randint(0, 255) << 8) + random.randint(0, 255)
    plot += k3d.points(ps, point_size=0.05, color=color)
plot.display()

(95857, 3)
(90919, 3)
(108105, 3)
(105119, 3)


/scratch/rhm4nj/.conda/ginn_env11/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [7]:
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree
from concurrent.futures import ThreadPoolExecutor

def average_nearest_neighbor_distance(points):
    tree = cKDTree(points)
    distances, _ = tree.query(points, k=2)
    nn_distances = distances[:, 1]
    return np.mean(nn_distances), np.std(nn_distances)

def remove_outliers(point_cloud, method="statistical", nb_neighbors=25, std_ratio=1.5, radius=0.1, min_neighbors=5, ret_indices=False):
    # Convert to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    if method == "statistical":
        # Statistical Outlier Removal (SOR)
        clean_pcd, inlier_indices = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    elif method == "radius":
        # Radius Outlier Removal (ROR)
        clean_pcd, inlier_indices = pcd.remove_radius_outlier(nb_points=min_neighbors, radius=radius)
    else:
        raise ValueError("Invalid method. Use 'statistical' or 'radius'.")

    if ret_indices:
        return np.asarray(clean_pcd.points), np.asarray(inlier_indices)

    return np.asarray(clean_pcd.points)

def filtered_point_cloud_indices(A, B, min_distance=0.1):
    tree = cKDTree(A)
    distances, _ = tree.query(B)
    return distances > min_distance

def sample_states_and_controls_timed(inner_point_cloud, point_cloud, N, K, min_dot=0.25):
    def sample_vector():
        vec = np.random.randn(3)
        return vec / np.linalg.norm(vec)

    idx = np.random.choice(point_cloud.shape[0], N, replace=False)
    sampled_points = point_cloud[idx]
    kdtree = cKDTree(inner_point_cloud)
    distances, nearest_indices = kdtree.query(sampled_points, k=2)
    nearest_neighbors = inner_point_cloud[nearest_indices[:, 1]]
    sampled_data = []

    for i in range(N):
        x, y, z = sampled_points[i]
        uc = nearest_neighbors[i] - sampled_points[i]
        uc = uc / np.linalg.norm(uc)
        sampled_data.append([x, y, z, uc[0], uc[1], uc[2]])
        for _ in range(K - 1):
            random_vector = sample_vector()
            while np.dot(random_vector, uc) < min_dot:
                random_vector = -random_vector
                if np.dot(random_vector, uc) < min_dot:
                    random_vector = sample_vector()
            sampled_data.append([x, y, z, random_vector[0], random_vector[1], random_vector[2]])

    return np.array(sampled_data)

def compute_interface_normals(interface_filtered, normal_radius, max_nn):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(interface_filtered)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=max_nn)
    )
    pcd.normalize_normals()
    normals = np.asarray(pcd.normals)
    print("Interface / Normals", interface_filtered.shape, normals.shape)
    return normals

def compute_pts_on_env(domain_filtered, all_outers_filtered, interface_density, pts_on_env_thickness):
    ao_env = all_outers_filtered[
        filtered_point_cloud_indices(domain_filtered, all_outers_filtered, interface_density)
    ]
    pts_on_env = ao_env[
        ~filtered_point_cloud_indices(domain_filtered, ao_env, pts_on_env_thickness)
    ]
    print("Done with pts_on_env", pts_on_env.shape)
    return pts_on_env

def compute_outer_points(domain_filtered, all_outers_filtered, outer_density):
    outers = all_outers_filtered[
        filtered_point_cloud_indices(domain_filtered, all_outers_filtered, outer_density)
    ]
    print("Done with outers/envelope", outers.shape)
    return outers

In [9]:
augemented_points = []

for partition_idx, partition in enumerate(partitions):
    print(points.shape)
    points = partition
    interface_thickness=0.25
    inner_thickness=0.05
    normal_radius=0.05
    max_nn=30

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=max_nn)
    )
    pcd.normalize_normals()
    normals = np.asarray(pcd.normals)

    outer_extras = []
    bounds, bounds_box = calculate_bounds(points)
    widths = bounds[:, 1] - bounds[:, 0]  # [x_width, y_width, z_width]
    outer_extras_thickness = np.max(widths)
    for step in np.linspace(1e-1, outer_extras_thickness, 60):
        sampled_indices = downsample_random_indices(points, 20_000)
        points_tmp = points[sampled_indices]
        normals_tmp = normals[sampled_indices]
        
        outer_extras.append(points_tmp + normals_tmp  * step)
        outer_extras.append(points_tmp - normals_tmp  * step)
    envelope = np.vstack(outer_extras)

    inners = []
    for step in np.linspace(1e-2, inner_thickness, 10):
        inners.append(points + normals * step)
        inners.append(points - normals * step)

    domain = np.vstack(inners)  # And the opposite direction
    inner_m, inner_std = average_nearest_neighbor_distance(domain)
    inner_density = inner_m + inner_std * 6

    interface_thickness = inner_thickness + 0.25
    outers = []
    normals_interface = []
    for step in np.linspace(interface_thickness, interface_thickness + 0.25 + inner_density):
        outers.append(points + normals  * step)
        outers.append(points - normals  * step)
    interface = np.vstack(outers)
    normals_interface = np.vstack([normals] * len(outers))

    pts_on_env_thickness = interface_thickness + 0.1
    pts_on_envs = []
    for step in np.linspace(pts_on_env_thickness, pts_on_env_thickness + 0.4 + inner_density):
        pts_on_envs.append(points + normals  * step)
        pts_on_envs.append(points - normals  * step)
    pts_on_env = np.vstack(pts_on_envs)

    min_neighbors = 8

    # inner layer
    domain_filtered = domain[downsample_random_indices(domain, 500_000 // len(partitions))]
    domain_filtered = remove_outliers(domain_filtered, radius=inner_density, min_neighbors=min_neighbors)
    inner_m, inner_std = average_nearest_neighbor_distance(domain_filtered)
    inner_density = inner_m + inner_std * 6 + 0.05
    print("Done with inner layer", domain_filtered.shape)

    # all points
    bbox_offset = 0.5
    n_all_outers = 3_000_000 // len(partitions)
    bounds, bounds_box = calculate_bounds(domain_filtered)
    expanded_bounds = bounds_box[0] - bbox_offset, bounds_box[1] + bbox_offset
    all_outers = np.random.uniform(low=expanded_bounds[0], high=expanded_bounds[1], size=(n_all_outers, 3))
    print("Done with all points", all_outers.shape)

    # interface
    interface_thickness = inner_density + 0.05
    all_outers_filtered = all_outers[
        filtered_point_cloud_indices(domain_filtered, all_outers, inner_density)
    ]
    interface_filtered = all_outers_filtered[
        ~filtered_point_cloud_indices(domain_filtered, all_outers_filtered, interface_thickness)
    ]
    print("Done with interface", interface_filtered.shape)

    # Prepare variables for parallel work
    interface_density = interface_thickness + inner_density
    pts_on_env_thickness = interface_density + 0.05
    outer_density = pts_on_env_thickness + inner_density

    # Run them in parallel
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(compute_interface_normals, interface_filtered, normal_radius, max_nn),
            executor.submit(compute_pts_on_env, domain_filtered, all_outers_filtered, interface_density, pts_on_env_thickness),
            executor.submit(compute_outer_points, domain_filtered, all_outers_filtered, outer_density)
        ]
        normals_filtered, pts_on_env_filtered, envelope_filtered = [f.result() for f in futures]

    # control
    N = 2000
    K = 40
    control_outs_env = sample_states_and_controls_timed(domain_filtered, envelope_filtered, N // 3, K)
    control_outs_on_env = sample_states_and_controls_timed(domain_filtered, pts_on_env_filtered, N // 3, K)
    control_outs_interface = sample_states_and_controls_timed(domain_filtered, interface_filtered, N // 3, K)
    control_outs = np.vstack([control_outs_env, control_outs_interface, control_outs_on_env])
    control_points, controls = control_outs[:, :3], control_outs[:, 3:]
    print("Done with control", control_points.shape)

    # bounds, scaling, translation
    all_points = np.vstack([domain, interface_filtered, pts_on_env_filtered, envelope_filtered])
    bounds_og, bounds_coords = calculate_bounds(all_points)
    bbox_min, bbox_max = bounds_coords
    bounds = bounds_og.copy()

    all_points_obj = np.vstack([domain, interface_filtered])
    bounds_obj, _ = calculate_bounds(all_points_obj)

    center_for_translation = (bbox_max + bbox_min) / 2
    scale_factor = max(bbox_max - bbox_min) / 2

    print("Done with bounds, scaling, translation:", bounds_coords, scale_factor, center_for_translation)

    augemented_points.append({
        "class": "room",
        "pts_inside": domain_filtered,
        "env_outside_pts": envelope_filtered,
        "pts_on_env": pts_on_env_filtered,
        "outside_points": envelope_filtered,
        "control_points": control_outs,
        "control_points_on_env": control_outs_on_env,
        "control_points_env": control_outs_env,
        "control_points_interface": control_outs_interface,
        "original": points,
        "interface_pts": interface_filtered,
        "interface_normals": normals_filtered,
        "bounds": bounds,
        "bounds_obj": bounds_obj,
        "scale_factor": scale_factor,
        "center_for_translation": center_for_translation
    })



(105119, 3)


KeyboardInterrupt: 

In [ ]:
import k3d
import numpy as np
import random

plot = k3d.plot()

colors = {
    "original": 0xff0000,             # Red
    "pts_inside": 0x00ff00,           # Green
    "interface_pts": 0x0000ff,        # Blue
    # "env_outside_pts": 0xffff00,      # Yellow
    "pts_on_env": 0xff00ff,           # Magenta
    # "outside_points": 0x00ffff,       # Cyan
    "control_points": 0x808080,       # Gray
    # "control_points_on_env": 0xFFA500,# Orange
    # "control_points_env": 0x800080,   # Purple
    # "control_points_interface": 0x008000 # Dark Green
}

# print(domain.shape, domain_normals.shape)
# # plot += k3d.vectors(domain[:domain_normals.shape[0]], domain_normals, color=0x0000ff, head_size=0.1)  # Blue
# plot += k3d.vectors(interface, normals, color=0xFFA500, head_size=0.1, line_width=0.001)  # Blue
# plot += k3d.points(interface_tmp, point_size=0.01, color=0x808080)
# plot += k3d.points(pts_on_env_tmp, point_size=0.01, color=0x008000)

# plot += k3d.points(bounds_obj.T, point_size=0.05, color=0x008000)
plot.display()
for i, data_dict in enumerate(augemented_points):
    print(f"Plotting object {i}: {data_dict.get('class', 'Unknown')}")
    
    for key, value in data_dict.items():
        if not isinstance(value, np.ndarray):
            continue
        
        if value.ndim == 2 and value.shape[1] >= 3:
            pts = value[:, :3]
            col = colors.get(key, -1)
            if col != -1:
                plot += k3d.points(pts, point_size=0.05, color=col)
                print(f"{key} bounds: min {pts.min(axis=0)}, max {pts.max(axis=0)}")
                print("ADDED", key, pts.shape)

# plot += k3d.points(interface_tmp_big, point_size=0.01, color=0x008000)

Output()

Plotting object 0: room0
pts_inside bounds: min [-0.92921836 -1.23335263 -1.572127  ], max [3.03439762 1.16676285 1.31849531]
ADDED pts_inside (116841, 3)
pts_on_env bounds: min [-1.25123055 -1.55077036 -1.88774415], max [3.34788299 1.48184972 1.63654972]
ADDED pts_on_env (50454, 3)
control_points bounds: min [-1.42622864 -1.73189162 -2.07102385], max [3.53160289 1.66048798 1.81518002]
ADDED control_points (79920, 3)
original bounds: min [-0.87930405 -1.18597937 -1.52736306], max [3.00215173 1.16263068 1.27448845]
ADDED original (95629, 3)
interface_pts bounds: min [-1.08604979 -1.38793394 -1.72917026], max [3.18750682 1.32266279 1.47060348]
ADDED interface_pts (47896, 3)
Plotting object 1: room1
pts_inside bounds: min [-0.90764763  1.15563714 -1.5730405 ], max [3.02892414 3.54621133 1.3026188 ]
ADDED pts_inside (116615, 3)
pts_on_env bounds: min [-1.22496583  0.84397361 -1.88728547], max [3.34527534 3.86194008 1.62134666]
ADDED pts_on_env (49579, 3)
control_points bounds: min [-1.4024

In [ ]:
import shutil

name = room_name + "_partitions"
out_path = os.path.join(
    "/scratch/rhm4nj/cral/cral-ginn/ginn/myvis/data_gen", 
    "replica",
    name
)

if not os.path.exists(out_path):
    os.makedirs(out_path)
else:
    shutil.rmtree(out_path)

skips_names = []

for idx, values in enumerate(augemented_points):
    folder_name = f"{idx}_{values['class'][:-1]}"
    folder_path = os.path.join(out_path, folder_name)
    os.makedirs(folder_path)

    for name, arrays in values.items():
        if name in skips_names: continue

        print(f'Saving to {folder_name}:', name)
        np.save(f'{folder_path}/{name}.npy', arrays)

Saving to 0_room: class
Saving to 0_room: pts_inside
Saving to 0_room: env_outside_pts
Saving to 0_room: pts_on_env
Saving to 0_room: outside_points
Saving to 0_room: control_points
Saving to 0_room: control_points_on_env
Saving to 0_room: control_points_env
Saving to 0_room: control_points_interface
Saving to 0_room: original
Saving to 0_room: interface_pts
Saving to 0_room: interface_normals
Saving to 0_room: bounds
Saving to 0_room: bounds_obj
Saving to 0_room: scale_factor
Saving to 0_room: center_for_translation
Saving to 1_room: class
Saving to 1_room: pts_inside
Saving to 1_room: env_outside_pts
Saving to 1_room: pts_on_env
Saving to 1_room: outside_points
Saving to 1_room: control_points
Saving to 1_room: control_points_on_env
Saving to 1_room: control_points_env
Saving to 1_room: control_points_interface
Saving to 1_room: original
Saving to 1_room: interface_pts
Saving to 1_room: interface_normals
Saving to 1_room: bounds
Saving to 1_room: bounds_obj
Saving to 1_room: scale_fa